In [ ]:
# Install required packages
%pip install opencv-python face-recognition scikit-learn

In [ ]:
# Import libraries
import cv2
import numpy as np
import os
from pathlib import Path
import face_recognition
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pickle

In [ ]:
# Configuration
WATCH_LIST_DIR = Path("../data/raw/faces/watch_list")
MODEL_OUTPUT_DIR = Path("../data/models/face_recognition")

print(f"Watch list directory: {WATCH_LIST_DIR}")
print(f"Model output directory: {MODEL_OUTPUT_DIR}")

## Load Training Data

Load face images from the known faces directory.

In [ ]:
# TODO: Implement face loading and embedding extraction
def load_watch_list_faces(faces_dir: Path):
    """Load faces and extract embeddings."""
    embeddings = []
    labels = []
    
    for person_dir in faces_dir.iterdir():
        if person_dir.is_dir():
            person_name = person_dir.name
            for image_path in person_dir.glob("*.jpg"):
                # Load image and extract embedding
                image = face_recognition.load_image_file(str(image_path))
                face_encodings = face_recognition.face_encodings(image)
                
                if face_encodings:
                    embeddings.append(face_encodings[0])
                    labels.append(person_name)
    
    return np.array(embeddings), np.array(labels)

# Load data
# embeddings, labels = load_watch_list_faces(WATCH_LIST_DIR)

## Train Classifier

Train an SVM classifier on the face embeddings.

In [ ]:
# TODO: Implement classifier training
def train_classifier(embeddings, labels):
    """Train SVM classifier for face recognition."""
    X_train, X_test, y_train, y_test = train_test_split(
        embeddings, labels, test_size=0.2, random_state=42
    )
    
    classifier = SVC(kernel='rbf', probability=True)
    classifier.fit(X_train, y_train)
    
    accuracy = classifier.score(X_test, y_test)
    print(f"Test accuracy: {accuracy:.2%}")
    
    return classifier

## Export Model

Save the trained model for deployment.

In [ ]:
# TODO: Save model
def save_model(classifier, labels, output_path):
    """Save the trained model and label encoder."""
    model_data = {
        'classifier': classifier,
        'labels': list(set(labels))
    }
    
    with open(output_path, 'wb') as f:
        pickle.dump(model_data, f)
    
    print(f"Model saved to: {output_path}")